# Uni-MuMER Project - Kaggle Setup

Notebook này cài đặt và chạy Uni-MuMER Project trên Kaggle với Python 3.10 và conda

Uni-MuMER là mô hình nhận diện ký tự toán học viết tay sử dụng Qwen2.5-VL-3B


## 1. Cài đặt Miniconda


In [ ]:
# Cài đặt Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /kaggle/working/miniconda
!rm Miniconda3-latest-Linux-x86_64.sh

# Thêm conda vào PATH
import os
os.environ['PATH'] = "/kaggle/working/miniconda/bin:" + os.environ['PATH']


## 2. Tạo môi trường Python 3.10


In [ ]:
# Tạo môi trường Python 3.10
!/kaggle/working/miniconda/bin/conda create -n unimumer python=3.10 -y


## 3. Kiểm tra phiên bản Python và pip


In [ ]:
# Kiểm tra phiên bản Python và pip
shell_script = """
source /kaggle/working/miniconda/bin/activate unimumer
python --version
pip --version
"""
with open("activate_env.sh", "w") as f:
    f.write(shell_script)
!bash activate_env.sh


## 4. Clone repository từ GitHub

**Lưu ý**: Thay `YOUR_USERNAME` bằng username GitHub của bạn


In [ ]:
# Clone repo (thay YOUR_USERNAME bằng username GitHub của bạn)
# !git clone https://github.com/YOUR_USERNAME/Nhan-Dien-Ky-Tu-Toan-Hoc.git

# Hoặc nếu đã có trong Kaggle dataset, copy từ đó
# !cp -r /kaggle/input/unimer-project/* /kaggle/working/


## 5. Di chuyển vào thư mục dự án


In [ ]:
# Di chuyển vào thư mục BaseLine
%cd "Nhan-Dien-Ky-Tu-Toan-Hoc/BaseLine"


## 6. Cài đặt PyTorch với CUDA


In [ ]:
# Cài đặt PyTorch với CUDA 12.4 (tương thích với Kaggle)
!source /kaggle/working/miniconda/bin/activate unimumer && conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch -c nvidia -y


## 7. Cài đặt các dependencies từ requirements_training.txt


In [ ]:
# Upgrade pip trước
!source /kaggle/working/miniconda/bin/activate unimumer && pip install --upgrade pip

# Cài đặt các gói từ requirements_training.txt
!source /kaggle/working/miniconda/bin/activate unimumer && pip install -r requirements_training.txt


## 8. Clone LLaMA-Factory nếu chưa có


In [ ]:
# Clone LLaMA-Factory vào thư mục train/
!cd train && git clone https://github.com/hiyouga/LLaMA-Factory.git

# Cài đặt LLaMA-Factory
!source /kaggle/working/miniconda/bin/activate unimumer && cd train/LLaMA-Factory && pip install -e .


## 9. Chuẩn bị dữ liệu


In [ ]:
# Giải nén dữ liệu nếu có trong repo
# !unzip data.zip -d .

# Hoặc tải từ Kaggle dataset (bỏ comment nếu sử dụng cách này)
# !cp /kaggle/input/unimer-data/data.zip .
# !unzip data.zip -d .


## 10. Kiểm tra cấu hình và GPU


In [ ]:
# Kiểm tra GPU và PyTorch
check_script = """
source /kaggle/working/miniconda/bin/activate unimumer
python -c "import torch; print('CUDA available:', torch.cuda.is_available()); print('GPU count:', torch.cuda.device_count()); print('GPU name:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A')"
"""
with open("check_gpu.sh", "w") as f:
    f.write(check_script)
!bash check_gpu.sh


## 11. Chạy training

**Lưu ý**: Đảm bảo đã có deepspeed config trong `train/LLaMA-Factory/examples/deepspeed/`


In [ ]:
# Chạy training với cấu hình Uni-MuMER
# Lưu ý: Đảm bảo đã có deepspeed config trong train/LLaMA-Factory/examples/deepspeed/
!source /kaggle/working/miniconda/bin/activate unimumer && cd train/LLaMA-Factory && llamafactory-cli train ../Uni-MuMER-train.yaml


## 12. Resume training từ checkpoint (nếu cần)

Nếu bạn đã train vài epoch trên local, có thể resume từ checkpoint


In [ ]:
# Resume từ checkpoint (thay XXXX bằng số checkpoint)
# !source /kaggle/working/miniconda/bin/activate unimumer && cd train/LLaMA-Factory && llamafactory-cli train ../Uni-MuMER-train.yaml --resume_from_checkpoint saves/qwen2.5_vl-3b/full/sft/standred/uni-mumer_full/checkpoint-XXXX


## 13. Lưu kết quả và môi trường

Lưu checkpoints và logs để có thể sử dụng trong các phiên tiếp theo


In [ ]:
# Lưu kết quả training
!mkdir -p /kaggle/output/unimer_results
!cp -r train/LLaMA-Factory/saves /kaggle/output/unimer_results/ 2>/dev/null || echo "No saves directory found"
!cp -r train/LLaMA-Factory/logs /kaggle/output/unimer_results/ 2>/dev/null || echo "No logs directory found"


## 14. Đánh giá mô hình sau khi train


In [ ]:
# Đánh giá trên các test set
# !source /kaggle/working/miniconda/bin/activate unimumer && bash eval/eval_all.sh -m saves/qwen2.5_vl-3b/full/sft/standred/uni-mumer_full -s test1 -b 32768
